**Introduction to Window Function**

Window functions applies aggregate and ranking functions over a particular window (set of rows). OVER clause is used with window functions to define that window. OVER clause does two things :

1. Partitions rows into form set of rows. (PARTITION BY clause is used)
1. Orders rows within those partitions into a particular order. (ORDER BY clause is used)

**Types of Window functions**

1. Aggregate Window Functions
1. Ranking Window Functions
1.Value Window Functions

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS testdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE testdb;'
!pip install psycopg2-binary

In [11]:
# Importing Python Libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [12]:
# Defining Postgres Connection
connection_string = {'host':'localhost',
                     'dbname':'testdb',
                     'user':'postgres',
                     'password':'postgres',
                     'port':5432}
connection = psycopg2.connect(**connection_string)

**Aggregate Window Functions**

1. Sum()

1. Max()

1. Min()

1. Avg()

1. Count()

In [13]:
sql_query = """
Create table purchase (
  customer_id INT Primary Key,
  customer_name varchar(100),
  city varchar(100),
  amount double precision,
  purchasing_date date
)
"""
with connection.cursor() as cur:
  cur.execute(sql_query)

In [14]:
insert_query = """ insert into purchase(customer_id, customer_name, city, amount, purchasing_date)
values (1,'Manish','Sonipat',1000,'2021/01/05'),
       (2,'Raj','Noida',2500,'2021/01/08'),
       (3,'Shivani','Gurugram',1000,'2021/01/08'),
       (4,'Rohan','Gurugram',1200,'2021/01/08'),
       (5,'Sonu','Delhi',2800,'2021/01/09'),
       (6,'Manoj','Hisar',2500,'2021/01/09'),
       (7,'Deepak','Hisar',2400,'2021/01/09'),
       (8,'Rohit','Gurugram',1000,'2021/01/10'),
       (9,'Rohan','Gurugram',1200,'2021/01/10'),
       (10,'Raj','Noida',3500,'2021/01/11'),
       (11,'Yogesh','Hisar',3000,'2021/01/11'),
       (12,'Raj','Noida',3000,'2021/01/12'),
       (13,'Manish','Sonipat',1500,'2021/01/12'),
       (14,'Shivani','Gurugram',2000,'2021/01/13'),
       (15,'Rohan','Gurugram',2500,'2021/01/13'),
       (16,'Deepak','Hisar',2000,'2021/01/13'),
       (17,'Sonu','Delhi',1500,'2021/01/14'),
       (18,'Raj','Noida',2000,'2021/01/14'),
       (19,'Yogesh','Hisar',3800,'2021/01/14')
"""
with connection.cursor() as cur:
  cur.execute(insert_query)

In [15]:
test_query = """ Select * from purchase;
"""
pd.read_sql_query(test_query,connection)

**Problem 1. Sum()**

Calculate total amount spent everyday and date should be in descending order. Also display every customer_id, name and city.

In [16]:
sql_query1 = """ Select customer_id, customer_name, city, purchasing_date,
                  sum(amount) over (partition by purchasing_date order by purchasing_date desc) as total_amount
                  from purchase;
"""
pd.read_sql_query(sql_query1,connection)

**Problem 2. Avg()**

Find average amount for each city and for each day.

In [20]:
Sql_query2 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 Avg(amount) over(partition by city, date_trunc('day',purchasing_date)) as average_per_city_and_date
                 from purchase;
"""
pd.read_sql_query(Sql_query2,connection)

**Problem 3. Min()**

Find minimum purchase amount for each city

In [23]:
Sql_query3 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 min(amount) over(partition by city) as min_per_city_and_date
                 from purchase;
"""
pd.read_sql_query(Sql_query3,connection)

**Problem 4. Max()**

Find maximum purchase amount for each city

In [24]:
Sql_query4 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 max(amount) over(partition by city) as max_per_city_and_date
                 from purchase;
"""
pd.read_sql_query(Sql_query4,connection)

**Use of Distinct is not allowed with over clause**

**Problem 4. Count()**

Find total number of purchase for each city

In [25]:
Sql_query5 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 count(*) over(partition by city) as count_per_city_and_date
                 from purchase;
"""
pd.read_sql_query(Sql_query5,connection)

**Ranking Window Functions**

Ranking functions will rank a specified field and categorize them according to their values.

**Problem 1. Count()**
Rank the purchasing amount in descending order.

Rank functions give same rank to equal values and skip next rank.

In [31]:
Sql_query6 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 rank() over(order by amount desc ) as r
                 from purchase;
"""
pd.read_sql_query(Sql_query6,connection)

**Problem 2. Dense_rank()**

It is similar to rank function but it does not skip rank.

In [30]:
Sql_query7 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 rank() over(order by amount desc ) as r
                 from purchase;
"""
pd.read_sql_query(Sql_query7,connection)

**Problem 3. Row_number()**

Row_number simply give row number of rows.

In [29]:
Sql_query8 = """ Select customer_id, customer_name, city, amount, purchasing_date,
                 rank() over(order by amount desc ) as r
                 from purchase;
"""
pd.read_sql_query(Sql_query8,connection)

Calculate top 2 purchasing amount for every city.

In [32]:
Sql_query9 = """ Select customer_id, customer_name, city, amount, purchasing_date,r from(
                 Select customer_id, customer_name, city, amount, purchasing_date,
                 rank() over(partition by city order by amount desc ) as r
                 from purchase)temp where r<3;
"""
pd.read_sql_query(Sql_query9,connection)

**Problem 4. Ntile()**

The SQL Server NTILE() is a window function that distributes rows of an ordered partition into a specified number of approximately equal groups, or buckets. It assigns each group a bucket number starting from one. For each row in a group, the NTILE() function assigns a bucket number representing the group to which the row belongs.

In [33]:
Sql_query10 = """ 
                 Select customer_id, customer_name, city, amount, purchasing_date,
                 ntile(4) over( order by amount desc ) as quartile
                 from purchase;
"""
pd.read_sql_query(Sql_query10,connection)

**Value Window Functions**

**Lag()**

Lag function allows us read data from pervious row in same result set without join

Problem 1. Find out previous customer_id to every customer_id with in same city

In [34]:
sql_query11 = """ Select customer_name, city, amount, purchasing_date, customer_id,
                  LAG(customer_id,1) over (partition by city order by purchasing_date) as pre_customer_id
                 from purchase
"""
pd.read_sql_query(sql_query11,connection)

**Lead()**

Lead function allows us read data from next row in same result set without join

Problem 2. Find out next customer_id to every customer_id with in same city

In [35]:
sql_query12 = """ Select customer_name, city, amount, purchasing_date, customer_id,
                  LEAD(customer_id,1) over (partition by city order by purchasing_date) as pre_customer_id
                 from purchase
"""
pd.read_sql_query(sql_query12,connection)

**First_value() and Last_value()**

These functions help you to identify first and last record within a partition or entire table if PARTITION BY is not specified.

Problem 3. Find out first and last customer name on every date. Also show other fields.

In [36]:
sql_query13 = """ Select customer_name, city, amount, purchasing_date, customer_id,
                  First_value(customer_name) over (partition by purchasing_date order by purchasing_date) as First_customer,
                  Last_value(customer_name) over (partition by purchasing_date order by purchasing_date) as Last_customer
                  from purchase
"""
pd.read_sql_query(sql_query13,connection)